# OmniSci Charting Example with Altair

Let's see if we can replicate [this](https://omnisci.github.io/mapd-charting/example/example1.html) OmniSci charting example in Python with Altair, Vega Lite, and Vega:

![](https://cloud.githubusercontent.com/assets/2932405/25641647/1acce1f2-2f4a-11e7-87d4-a4e80cb262f5.gif)

First, import `altair`, `ibis`, and `ibis_vega_transform` and connect to the database:

In [ ]:
import altair as alt
import ibis_vega_transform

import warnings
try:
    from ibis.backends import omniscidb as ibis_omniscidb
except ImportError as msg:
    warnings.warn(str(msg))
    from ibis import omniscidb as ibis_omniscidb

conn = ibis_omniscidb.connect(
    host='metis.mapd.com', user='demouser', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)

In [ ]:
conn.list_tables()

We can access the flights table:

In [ ]:
t = conn.table("flights_donotmodify")

In [ ]:
t

It has 7 million rows:

In [ ]:
t.info()

In [ ]:
t.head().execute()

## States

Now let's make the top left graph, the flights by state, letting you select one:

In [ ]:
states = alt.selection_multi(fields=['origin_state'])

In [ ]:
ibis_vega_transform.enable_debug()

In [ ]:
flights_by_state = alt.Chart(
    t,
    title="Total Number of Flights by State"
).mark_bar().encode(
    x='count()',
    y=alt.Y(
        'origin_state',
        sort=alt.Sort(encoding='x', order='descending')
    ),
    color=alt.condition(states, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
).add_selection(
    states
)
flights_by_state

Now let's add the little text there with the count as well.

In [ ]:
ibis_vega_transform.disable_debug()

In [ ]:
flights_by_state_text = alt.Chart(t).mark_text(dx=20).encode(
    x='count()',
    y=alt.Y('origin_state', sort=alt.Sort(encoding='x', order='descending')),
    text='count()'
)
flights_by_state_text

We can combine them together now:

In [ ]:
def create_flights_by_state(map_fn=lambda c: c):
    return map_fn(flights_by_state_text) + map_fn(flights_by_state)

In [ ]:
create_flights_by_state()

## Delay chart

We do the same now for the delay chart:

In [ ]:
airlines = alt.selection_multi(fields=['carrier_name'])

def create_carrier_delay(before_aggregate_fn=lambda c: c):
    carrier_delay = before_aggregate_fn(alt.Chart(
        t,
        title="Carrier Departure Delay by Arrival Delay (Minutes)"
    )).transform_aggregate(
        depdelay='mean(depdelay)',
        arrdelay='mean(arrdelay)',
        groupby=["carrier_name"]
    ).mark_point(filled=True, size=200).encode(
        x='depdelay',
        y='arrdelay',
        color=alt.condition(airlines, alt.ColorValue("steelblue"), alt.ColorValue("grey")),
        tooltip=['carrier_name', 'depdelay', 'arrdelay']
    ).add_selection(airlines)

    carrier_delay_text = before_aggregate_fn(alt.Chart(
        t
    )).transform_aggregate(
        depdelay='mean(depdelay)',
        arrdelay='mean(arrdelay)',
        groupby=["carrier_name"]
    ).mark_text().encode(
        x='depdelay',
        y='arrdelay',
        text='carrier_name',
    )
    return carrier_delay + carrier_delay_text

create_carrier_delay()

## States + Delays

Now we can combine these two charts, by adding filters by each others selections:

In [ ]:
(
    create_flights_by_state(lambda c: c.transform_filter(airlines)) |
    create_carrier_delay(lambda c: c.transform_filter(states))
)

In [ ]:
ibis_vega_transform.disable_debug()

## Departure Time

Now we can add the final part, the depature time chart:

In [ ]:
dates = alt.selection_interval(
    fields=['dep_timestamp'],
    encodings=['x'],
)


def create_time(map_fn=lambda c:c):
    return map_fn(
        alt.Chart(
            t,
            title='Number of Flights by Departure Time'
        )
    ).transform_filter(
        'datum.dep_timestamp != null'
    ).mark_line().encode(
        alt.X(
            'yearmonthdate(dep_timestamp):T',
        ),
        alt.Y(
            'count():Q',
            scale=alt.Scale(zero=False)
        )
    ).add_selection(
        dates
    )
create_time()

## Combined

And add them all together!

In [ ]:
HEIGHT = 800
WIDTH = 1000

(
        (
        create_flights_by_state(
            lambda c: c.transform_filter(
                {"and": [airlines, dates]}
            ).properties(
                height= 2 * HEIGHT / 3,
                width=WIDTH / 2
            )
        ) | 
        create_carrier_delay(
            lambda c: c.transform_filter(
                {"and": [dates, states]}
            )
        ).properties(
            height=2 * HEIGHT / 3,
            width=WIDTH / 2
        )
    ) & (
        create_time(
            lambda c: c.transform_filter(
                {"and": [airlines, states]}
            ).properties(
                height=HEIGHT / 3,
                width=WIDTH + 50
            )
        )
    )
).configure_axis(
    grid=False
).configure_view(
    strokeOpacity=0
).configure(
    autosize={
        'resize': True
    }
)